In [8]:
import os
import json
from openai import OpenAI
from dotenv import load_dotenv
from pathlib import Path

# Load environment variables
load_dotenv()

# Initialize OpenAI client with API key from .env
api_key = os.getenv('OPENAI_API_KEY')
if not api_key:
    raise ValueError("Please set OPENAI_API_KEY in your .env file")
client = OpenAI(api_key=api_key)

# Path to dataset folder and metadata
dataset_path = Path("dataset")
metadata_file = dataset_path / "metadata.jsonl"
new_metadata_file = dataset_path / "metadata_with_new_prompts.jsonl"

# Read existing metadata
with open(metadata_file, 'r') as f:
    metadata_entries = [json.loads(line) for line in f]

# Process each entry
for entry in metadata_entries:
    image_path = dataset_path / entry['file_name']
    
    # Encode image to base64
    with open(image_path, 'rb') as image_file:
        import base64
        base64_image = base64.b64encode(image_file.read()).decode('utf-8')
    
    try:
        # Generate description using GPT-4 Vision
        response = client.chat.completions.create(
            model="gpt-4o-mini",  # corrected model name
            messages=[
                {
                    "role": "user",
                    "content": [
                        {"type": "text", "text": "Describe this image in detail, focusing on the facial expression and emotional content. If the file name contains 'happy' please include that in the description. The same goes for if the file name contains 'sad'"},
                        {
                            "type": "image_url",
                            "image_url": {
                                "url": f"data:image/jpeg;base64,{base64_image}"
                            }
                        }
                    ]
                }
            ],
            max_tokens=300
        )
        
        
        # Update prompt in entry
        entry['prompt'] = response.choices[0].message.content
        
        # Write to new file
        with open(new_metadata_file, 'a') as f:
            json.dump(entry, f)
            f.write('\n')
            
        print(f"Processed {entry['file_name']}")
        print(f"New prompt: {entry['prompt']}")
        
    except Exception as e:
        print(f"Error processing {entry['file_name']}: {str(e)}")

print("Processing complete!")


Processed sad_01.jpg
New prompt: The image features a minimalist face made using simple characters, set against a solid blue background. The facial expression consists of two large, rounded eyes and a small, open mouth, which can convey a sense of innocence or curiosity.

The overall emotional content appears playful and lighthearted. The configuration of the characters suggests a cute or whimsical demeanor, which may evoke feelings of happiness or joy. The absence of any harsh lines or complex features enhances this cheerful impression. 

Since the filename does not suggest a specific emotional tone, I describe it based on the visual elements alone.
Processed sad_02.jpg
New prompt: The image consists of a sequence of characters that conveys a facial expression. It features an emoticon-style representation that suggests a confused or frustrated emotional state. The combination of characters—specifically the use of symbols like "#" and the expression "<"—creates a visually distinct expr

In [9]:
# create new jsonl from dataset/metadata_with_new_prompts.jsonl
# detect if each jsonl file_name has regex 'happy' or 'sad' (e.g. sad_01.jpg or sad0020.png are both 'sad') 
# if 'happy' add the following to the beginning of the "prompt" field: "This is an image of a happy emoji, and what follows is a description of the emoji. "
# if 'sad' add the following to the beginning of the "prompt" field: "This is an image of a sad emoji, and what follows is a description of the emoji. "
# write the new jsonl to dataset/metadata_with_new_prompts_with_happy_sad.jsonl

import json
import re

# Input and output files
input_file = 'dataset/metadata_with_new_prompts.jsonl'
output_file = 'dataset/metadata_with_new_prompts_with_happy_sad.jsonl'

# Process each line
with open(input_file, 'r') as infile, open(output_file, 'w') as outfile:
    for line in infile:
        entry = json.loads(line)
        
        # Check if filename contains 'happy' or 'sad'
        if re.search(r'happy', entry['file_name'], re.IGNORECASE):
            entry['prompt'] = "This is an image of a happy emoji, and what follows is a description of the emoji. " + entry['prompt']
        elif re.search(r'sad', entry['file_name'], re.IGNORECASE):
            entry['prompt'] = "This is an image of a sad emoji, and what follows is a description of the emoji. " + entry['prompt']
            
        # Write modified entry to output file
        json.dump(entry, outfile)
        outfile.write('\n')

print(f"Processing complete! New file written to {output_file}")


Processing complete! New file written to dataset/metadata_with_new_prompts_with_happy_sad.jsonl


In [10]:
import json

input_file = 'dataset/metadata_with_new_prompts_with_happy_sad.jsonl'
output_file = 'dataset/metadata_with_new_prompts_without_last_sentence.jsonl'

with open(input_file, 'r') as infile, open(output_file, 'w') as outfile:
    for line in infile:
        entry = json.loads(line)
        
        # Split prompt into sentences and remove the last one
        sentences = entry['prompt'].split('. ')
        entry['prompt'] = '. '.join(sentences[:-1]) + '.'
        
        # Write modified entry to output file
        json.dump(entry, outfile)
        outfile.write('\n')

print(f"Processing complete! New file written to {output_file}")


Processing complete! New file written to dataset/metadata_with_new_prompts_without_last_sentence.jsonl


In [3]:
import json
import os
from openai import OpenAI
from pathlib import Path
from dotenv import load_dotenv
from tqdm import tqdm

# Load environment variables
load_dotenv()

# Initialize OpenAI client with API key from .env
api_key = os.getenv('OPENAI_API_KEY')
if not api_key:
    raise ValueError("Please set OPENAI_API_KEY in your .env file")
client = OpenAI(api_key=api_key)

input_file = 'dataset/metadata_with_new_prompts_without_last_sentence.jsonl'
output_file = 'dataset/metadata_with_shortened_prompts.jsonl'

# Count total lines
total_lines = sum(1 for _ in open(input_file))

# Process each entry
with open(input_file, 'r') as infile, open(output_file, 'w') as outfile:
    for line in tqdm(infile, total=total_lines, desc="Processing prompts"):
        entry = json.loads(line)
        
        # Get shorter summary from GPT
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {
                    "role": "user", 
                    "content": [
                        {
                            "type": "text",
                            "text": f"Summarize this text in 23 words or less: {entry['prompt']}"
                        }
                    ]
                }
            ],
            max_tokens=50
        )
        
        # Update prompt with shortened version
        entry['prompt'] = response.choices[0].message.content
        
        # Print the new prompt
        print(f"\nNew prompt for {entry['file_name']}:")
        print(entry['prompt'])
        
        # Write modified entry to output file
        json.dump(entry, outfile)
        outfile.write('\n')

print(f"\nProcessing complete! New file written to {output_file}")

Processing prompts:   1%|          | 1/102 [00:02<03:26,  2.05s/it]


New prompt for sad_01.jpg:
A cheerful emoji against a blue background expresses joy with raised eyebrows, wide eyes, and a smiling mouth, suggesting positivity.


Processing prompts:   2%|▏         | 2/102 [00:03<02:54,  1.75s/it]


New prompt for sad_02.jpg:
The emoji image depicts a frustrated expression using characters, with a frown suggesting displeasure, absent direct 'happy' or 'sad' indicators.


Processing prompts:   3%|▎         | 3/102 [00:04<02:13,  1.35s/it]

...


New prompt for happy0044.png:
A shiny gold trophy symbolizes achievement and joy, reflecting light prominently, suggesting celebratory moments despite lacking facial expressions.


Processing prompts:  99%|█████████▉| 101/102 [01:44<00:01,  1.33s/it]


New prompt for happy0045.png:
The text describes a welcoming emoji with medium brown skin, friendly eyes, a slight smile, and an eager gesture in a casual outfit.


Processing prompts: 100%|██████████| 102/102 [01:49<00:00,  1.07s/it]


New prompt for happy0046.png:
The cartoon cow emoji is cheerful, with big black eyes and a broad smile creating a warm, friendly expression.

Processing complete! New file written to dataset/metadata_with_shortened_prompts.jsonl
